In [32]:
import os
import shutil
import cv2
from moviepy.editor import VideoFileClip
import time
import numpy as np

def create_directories(base_path, dir_names):
    for dir_name in dir_names:
        os.makedirs(os.path.join(base_path, dir_name), exist_ok=True)

def move_video(video_path, target_dir):
    folder_name_component = ''.join(video_path.split('/')[-2].split('PREMIER')[0].split(' '))
    filename = os.path.basename(video_path)
    new_filename = f"{folder_name_component}_{filename}"
    target_path = os.path.join(target_dir, new_filename)
    shutil.move(video_path, target_path)



def display_video_and_get_label(video_path):
    clip = VideoFileClip(video_path)
    quit_pressed = False
    frame_counter = 0  # Initialize a frame counter

    for frame in clip.iter_frames(fps=24, dtype="uint8"):
        if frame_counter % 3 == 0:  # Display only even-numbered frames
            cv2.imshow('Video', cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
            key = cv2.waitKey(25) & 0xFF
            if key == ord('q'):  # Check if 'q' is pressed
                quit_pressed = True
                break
            elif key in [48, 49, 50, 51, 52]:  # ASCII codes for 0, 1, 2, 3, 4
                return chr(key)

        frame_counter += 1  # Increment the frame counter

    end_screen_time = 0.5  # Duration for 3 frames at 24 fps
    end_time = time.time() + end_screen_time
    while time.time() < end_time:
        black_screen = np.zeros((clip.size[1], clip.size[0], 3), dtype=np.uint8)
        cv2.putText(black_screen, 'end', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.imshow('Video', black_screen)
        cv2.waitKey(1)

    cv2.destroyAllWindows()
    clip.close()

    if quit_pressed:
        return 'q'  # Return 'q' to indicate that the program should quit

    return None

def process_folder(path):
    video_directory = path
    video_files = [os.path.join(video_directory, f) for f in os.listdir(video_directory) if f.endswith('.mp4')]

    base_path = 'labeled_videos'
    dir_names = ['0', '1', '2', '3', '4']
    create_directories(base_path, dir_names)

    for video in video_files:
        label = None
        while not label:  # Keep showing the video until a label is received
            label = display_video_and_get_label(video)

            if label == 'q':
                return False  # Return False to indicate that the program should quit

        # Once a label is received, move the video
        target_dir = os.path.join(base_path, label)
        move_video(video, target_dir)

    return True  # Return True to indicate successful completion
        
def list_subdirectories(directory):
    return [os.path.join(directory, d) for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]


def main():
    base_path = 'labeled_videos'
    dir_names = ['0', '1', '2', '3', '4']
    create_directories(base_path, dir_names)

    test_directory = 'test'
    subdirectories = list_subdirectories(test_directory)

    for subdirectory in subdirectories:
        should_continue = process_folder(subdirectory)
        if not should_continue:
            break  # Exit the loop (and thus the script) if 'q' was pressed




In [33]:
main()